# Model Training

## Import Libraries

In [1]:
import numpy as np
import os, torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import transforms
from models.project_models import FC_500_150, LeNet_CIFAR, LeNet_MNIST, Net
from utils.project_utils import get_clip_bounds, model_train, model_eval

## CIFAR-10 SSL Fix

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## Set Random Seed & Device

In [3]:
rand_seed = 100
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

## Download & Pre-Process Datasets

### MNIST Dataset

In [4]:
mnist_mean = 0.5
mnist_std = 0.5
mnist_dim = 28

mnist_min, mnist_max = get_clip_bounds(mnist_mean,
                                       mnist_std,
                                       mnist_dim)
mnist_min = mnist_min.to(device)
mnist_max = mnist_max.to(device)

mnist_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=0.0,
        std=np.divide(1.0, mnist_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, mnist_std),
        std=1.0)])

mnist_temp = datasets.MNIST(root='datasets/mnist', train=True,
                            download=True, transform=mnist_tf_train)
mnist_train, mnist_val = random_split(mnist_temp, [50000, 10000])

mnist_test = datasets.MNIST(root='datasets/mnist', train=False,
                            download=True, transform=mnist_tf)

### CIFAR-10 Dataset

In [5]:
cifar_mean = [0.491, 0.482, 0.447]
cifar_std = [0.202, 0.199, 0.201]
cifar_dim = 32

cifar_min, cifar_max = get_clip_bounds(cifar_mean,
                                       cifar_std,
                                       cifar_dim)
cifar_min = cifar_min.to(device)
cifar_max = cifar_max.to(device)

cifar_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_train = transforms.Compose([
    transforms.RandomCrop(
        size=cifar_dim,
        padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=np.divide(1.0, cifar_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, cifar_mean),
        std=[1.0, 1.0, 1.0])])

cifar_temp = datasets.CIFAR10(root='datasets/cifar-10', train=True,
                              download=True, transform=cifar_tf_train)
cifar_train, cifar_val = random_split(cifar_temp, [40000, 10000])

cifar_test = datasets.CIFAR10(root='datasets/cifar-10', train=False,
                              download=True, transform=cifar_tf)

cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


## Dataset Loaders

In [6]:
batch_size = 128
workers = 4

mnist_loader_train = DataLoader(mnist_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
mnist_loader_val = DataLoader(mnist_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
mnist_loader_test = DataLoader(mnist_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

cifar_loader_train = DataLoader(cifar_train, batch_size=batch_size,
                                shuffle=True, num_workers=workers)
cifar_loader_val = DataLoader(cifar_val, batch_size=batch_size,
                              shuffle=False, num_workers=workers)
cifar_loader_test = DataLoader(cifar_test, batch_size=batch_size,
                               shuffle=False, num_workers=workers)

## Training Parameters

In [7]:
train_model = True

epochs = 50

lr = 0.004
lr_nin = 0.01

momentum = 0.9

print_step = 5

if not os.path.isdir('weights/clean'):
    os.makedirs('weights/clean', exist_ok=True)

## MNIST Training

### LeNet Model

In [8]:
model = LeNet_MNIST().to(device)

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step)
    torch.save(model.state_dict(), 'weights/clean/mnist_lenet.pth')

model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth'))
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.5429,  Train Loss : 1.4682
      Val Acc : 0.8185,    Val Loss : 0.5283
Epoch [5]
    Train Acc : 0.9600,  Train Loss : 0.1284
      Val Acc : 0.9596,    Val Loss : 0.1247
Epoch [10]
    Train Acc : 0.9770,  Train Loss : 0.0729
      Val Acc : 0.9747,    Val Loss : 0.0830
Epoch [15]
    Train Acc : 0.9846,  Train Loss : 0.0494
      Val Acc : 0.9800,    Val Loss : 0.0664
Epoch [20]
    Train Acc : 0.9880,  Train Loss : 0.0378
      Val Acc : 0.9796,    Val Loss : 0.0664
Epoch [25]
    Train Acc : 0.9903,  Train Loss : 0.0304
      Val Acc : 0.9847,    Val Loss : 0.0545
Epoch [30]
    Train Acc : 0.9920,  Train Loss : 0.0245
      Val Acc : 0.9841,    Val Loss : 0.0580
Epoch [35]
    Train Acc : 0.9938,  Train Loss : 0.0196
      Val Acc : 0.9786,    Val Loss : 0.0772
Epoch [40]
    Train Acc : 0.9950,  Train Loss : 0.0156
      Val Acc : 0.9861,    Val Loss : 0.0552
Epoch [45]
    Train Acc : 0.9962,  Train Loss : 0.0123
      Val Acc : 0.9846,    Val Loss :

### FC Model

In [9]:
model = FC_500_150().to(device)

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              mnist_loader_train, mnist_loader_val,
                              print_step)
    torch.save(model.state_dict(), 'weights/clean/mnist_fc.pth')

model.load_state_dict(torch.load('weights/clean/mnist_fc.pth'))
_, _ = model_eval(device, model, mnist_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.6948,  Train Loss : 1.0164
      Val Acc : 0.8197,    Val Loss : 0.5555
Epoch [5]
    Train Acc : 0.9103,  Train Loss : 0.2975
      Val Acc : 0.9194,    Val Loss : 0.2674
Epoch [10]
    Train Acc : 0.9480,  Train Loss : 0.1723
      Val Acc : 0.9454,    Val Loss : 0.1731
Epoch [15]
    Train Acc : 0.9625,  Train Loss : 0.1231
      Val Acc : 0.9519,    Val Loss : 0.1534
Epoch [20]
    Train Acc : 0.9708,  Train Loss : 0.0963
      Val Acc : 0.9550,    Val Loss : 0.1481
Epoch [25]
    Train Acc : 0.9770,  Train Loss : 0.0759
      Val Acc : 0.9625,    Val Loss : 0.1227
Epoch [30]
    Train Acc : 0.9802,  Train Loss : 0.0633
      Val Acc : 0.9659,    Val Loss : 0.1117
Epoch [35]
    Train Acc : 0.9848,  Train Loss : 0.0516
      Val Acc : 0.9674,    Val Loss : 0.1044
Epoch [40]
    Train Acc : 0.9873,  Train Loss : 0.0419
      Val Acc : 0.9695,    Val Loss : 0.1049
Epoch [45]
    Train Acc : 0.9905,  Train Loss : 0.0329
      Val Acc : 0.9709,    Val Loss :

## CIFAR-10 Training

### Network-In-Network Model

In [10]:
model = Net().to(device)

model.load_state_dict(torch.load('weights/clean/cifar_nin.pth'))
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Evaluation (Clean Images)
     Test Acc : 0.9072,   Test Loss : 0.3178


### LeNet Model

In [11]:
model = LeNet_CIFAR().to(device)

if train_model:
    opt = torch.optim.SGD(model.parameters(),
                          lr=lr,
                          momentum=momentum)
    _, _, _, _, = model_train(device, model, opt, epochs,
                              cifar_loader_train, cifar_loader_val,
                              print_step)
    torch.save(model.state_dict(), 'weights/clean/cifar_lenet.pth')

model.load_state_dict(torch.load('weights/clean/cifar_lenet.pth'))
_, _ = model_eval(device, model, cifar_loader_test)

if device.type == 'cuda':
    torch.cuda.empty_cache()

Epoch [1]
    Train Acc : 0.1071,  Train Loss : 2.3005
      Val Acc : 0.1836,    Val Loss : 2.2926
Epoch [5]
    Train Acc : 0.4247,  Train Loss : 1.5375
      Val Acc : 0.4516,    Val Loss : 1.4806
Epoch [10]
    Train Acc : 0.5864,  Train Loss : 1.1482
      Val Acc : 0.5953,    Val Loss : 1.1451
Epoch [15]
    Train Acc : 0.6831,  Train Loss : 0.8944
      Val Acc : 0.6803,    Val Loss : 0.9022
Epoch [20]
    Train Acc : 0.7337,  Train Loss : 0.7613
      Val Acc : 0.7226,    Val Loss : 0.7958
Epoch [25]
    Train Acc : 0.7623,  Train Loss : 0.6751
      Val Acc : 0.7392,    Val Loss : 0.7455
Epoch [30]
    Train Acc : 0.7895,  Train Loss : 0.6058
      Val Acc : 0.7559,    Val Loss : 0.6991
Epoch [35]
    Train Acc : 0.8079,  Train Loss : 0.5526
      Val Acc : 0.7700,    Val Loss : 0.6746
Epoch [40]
    Train Acc : 0.8214,  Train Loss : 0.5085
      Val Acc : 0.7729,    Val Loss : 0.6707
Epoch [45]
    Train Acc : 0.8366,  Train Loss : 0.4676
      Val Acc : 0.7776,    Val Loss :